In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import statistics
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
pd.options.plotting.backend = "plotly"

## Fazendo modelos sem os dados de estatística

In [ ]:
df_games = pd.read_csv('../input/campeonato-brasileiro-de-futebol/campeonato-brasileiro-full.csv', sep=";")
df_games.head()

In [ ]:
df_ref_games = df_games

## Tratamento dos dados

In [ ]:
rows_of_day = []
rows_of_month = []
rows_of_year = []
for game in range(0,len(df_ref_games)):
    date = df_ref_games.iloc[game]["Data"]
    try:
        date = date.replace('/', '-')
        day = int(date.split('-')[2])
        month = int(date.split('-')[1])
        year = int(date.split('-')[0])
        rows_of_day.append(day)
        rows_of_month.append(month)
        rows_of_year.append(year)
    except:
        print(f'deu ruim na linha: {game}')
df_new_days = pd.DataFrame(rows_of_day, columns=['Data'])
df_new_months = pd.DataFrame(rows_of_month, columns=['Mês'])
df_new_years = pd.DataFrame(rows_of_year, columns=['Ano'])
df_ref_games['Data'] = df_new_days
df_ref_games = pd.concat([df_ref_games, df_new_months], axis=1)
df_ref_games = pd.concat([df_ref_games, df_new_years], axis=1)
df_ref_games.head()

### Retirando a informação da pontuação de cada time na época que ocorreu o jogo

In [ ]:
rows_of_home_points = []
rows_of_away_points = []
for ano in range(2000, 2022):
    df_year = df_ref_games[df_ref_games['Ano']==ano]
    '''a cada jogo vai percorrer todos os jogos do ano, 
    anteriores ao jogo atual, e contar quantas vezes o nome 
    apareceu em "Vencedor" e multiplicar por 3 (vitoria vale 3) e quantas vezes 
    apareceu "-" (empate vale 1) e somar'''
    for game in range(0, len(df_year)):
        rodada = str(df_year.iloc[game]['Rodada'])
        home_points = 0
        away_points = 0
        if rodada != '1':
            home_team = df_year.iloc[game]['Mandante']
            away_team = df_year.iloc[game]['Visitante']
            #pegar só jogos antes do que estamos analisando
            df_actual_range = df_year[:game+1]
            #separar apenas do time mandante
            df_home_in_home_games = df_actual_range[df_actual_range['Mandante'] == home_team]
            df_home_in_away_games = df_actual_range[df_actual_range['Visitante'] == home_team]
            #contar vitorias e empates do mandante
            vic_home_in_home = len(df_home_in_home_games[df_home_in_home_games['Vencedor'] == home_team])
            vic_home_out_home = len(df_home_in_away_games[df_home_in_away_games['Vencedor'] == home_team])
            home_ties_in_home = len(df_home_in_home_games[df_home_in_home_games['Vencedor'] == '-'])
            home_ties_out_home = len(df_home_in_away_games[df_home_in_away_games['Vencedor'] == '-'])
            home_points = vic_home_in_home*3 + vic_home_out_home*3 + home_ties_in_home + home_ties_out_home
            
            #separar apenas do time visitante
            df_away_in_home_games = df_actual_range[df_actual_range['Mandante'] == away_team]
            df_away_in_away_games = df_actual_range[df_actual_range['Visitante'] == away_team]
            #contar vitorias e empates do mandante
            vic_away_in_home = len(df_away_in_home_games[df_away_in_home_games['Vencedor'] == away_team])
            vic_away_out_home = len(df_away_in_away_games[df_away_in_away_games['Vencedor'] == away_team])
            away_ties_in_home = len(df_away_in_home_games[df_away_in_home_games['Vencedor'] == '-'])
            away_ties_out_home = len(df_away_in_away_games[df_away_in_away_games['Vencedor'] == '-'])
            away_points = vic_away_in_home*3 + vic_away_out_home*3 + away_ties_in_home + away_ties_out_home
        rows_of_home_points.append(home_points)
        rows_of_away_points.append(away_points)

In [ ]:
df_home_points = pd.DataFrame(rows_of_home_points, columns=['Pontos Mandante'])
df_away_points = pd.DataFrame(rows_of_away_points, columns=['Pontos Visitante'])
df_games_w_points= pd.concat([df_ref_games, df_home_points], axis=1)
df_games_w_points = pd.concat([df_games_w_points, df_away_points], axis=1)
df_games_w_points

### Alterando horário de formato string em int

In [ ]:
import math
rows_of_hours = []
rows_of_min = []
for game in range(0,len(df_games_w_points)):
    time = df_games_w_points.iloc[game]["Horário"]
    try:
        time = str(time)
        time = time.replace('h',':')
        hour = int(time.split(':')[0])
        minute = int(time.split(':')[1])
        rows_of_hours.append(hour)
        rows_of_min.append(minute)
    except:
        rows_of_hours.append(19)
        rows_of_min.append(0)
df_new_hours = pd.DataFrame(rows_of_hours, columns=['Hora'])
df_new_minutes = pd.DataFrame(rows_of_min, columns=['Minuto'])
df_games_w_points['Horário'] = df_new_hours
df_games_w_points = pd.concat([df_games_w_points, df_new_minutes], axis=1)
df_games_w_points.head()

### Transformando o resultado em inteiros

In [ ]:
rows_of_results = []
for game in range(0,len(df_games_w_points)):
    the_winner = 0
    winner = df_games_w_points.iloc[game]["Vencedor"]
    home_team = df_games_w_points.iloc[game]["Mandante"]
    away_team = df_games_w_points.iloc[game]["Visitante"]
    if winner == '-':
        the_winner = 3
    elif winner == home_team:
        the_winner = 1
    else:
        the_winner = 2
    rows_of_results.append(the_winner)
df_new_results = pd.DataFrame(rows_of_results, columns=['Vencedor'])
df_games_w_points['Vencedor'] = df_new_results
df_games_w_points.head()

In [ ]:
df_games_w_points.shape

In [ ]:
8338 - 8338*20/100

### Separando os datasets em treinos (80%) e testes (20%), sendo que por se tratar de dados com relação temporal, utilizaremos em ordem cronológica, com os 20% sendo o nosso "futuro"

In [ ]:
df_x_train = df_games_w_points[:6500].drop(["Vencedor", "Mandante Placar", "Visitante Placar", "Estado Vencedor"], axis=1)
df_ref = df_x_train
df_y_train_winner = df_games_w_points['Vencedor'][:6500]

df_x_test = df_games_w_points[6500:].drop(["Vencedor", "Mandante Placar", "Visitante Placar", "Estado Vencedor"], axis=1)
df_y_test_winner = df_games_w_points['Vencedor'][6500:]

df_y_train_home_goals = df_games_w_points['Mandante Placar'][:6500]
df_y_test_home_goals = df_games_w_points['Mandante Placar'][6500:]
df_y_train_away_goals = df_games_w_points["Visitante Placar"][:6500]
df_y_test_away_goals = df_games_w_points["Visitante Placar"][6500:]

## Processamento dos dados

In [ ]:
from sklearn import preprocessing 

df_text = df_ref.select_dtypes(include=['object'])

ohe = preprocessing.OneHotEncoder(handle_unknown='ignore', sparse=False)
ohe.fit(df_text)

X_text = ohe.transform(df_text)

df_text = pd.DataFrame(X_text, columns=ohe.get_feature_names(df_text.columns))
df_text.shape

In [ ]:
df_num = df_ref.select_dtypes(exclude=['object'])
print(df_num.shape)
df_proc = pd.concat([df_num.reset_index(drop=True), df_text.reset_index(drop=True)], axis=1)
df_proc

In [ ]:
# Normalização do dataset
ss = preprocessing.StandardScaler()
X_train = ss.fit_transform(df_proc)

X_train = pd.DataFrame(X_train, columns=df_proc.columns)
X_train.head()

In [ ]:
from sklearn.decomposition import PCA

# Verificando a importancia de cada dimensão
pca = PCA()
pca.fit(X_train)
features = range(pca.n_components_)

import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x = list(features),
        y = 1 - pca.explained_variance_ratio_
    )
)

fig.show()

In [ ]:
pca = PCA(n_components=286)
X_train_red = pca.fit_transform(X_train)
X_train_red.shape

In [ ]:
df_text_test = df_x_test.select_dtypes(include=['object'])

X_text_test = ohe.transform(df_text_test)

df_text_test = pd.DataFrame(X_text_test, columns=ohe.get_feature_names(df_text_test.columns))

# ####
df_num_test = df_x_test.select_dtypes(exclude=['object'])
df_proc_test = pd.concat([df_num_test.reset_index(drop=True), df_text_test.reset_index(drop=True)], axis=1)

# Normalização do dataset
ss = preprocessing.StandardScaler()
X_test = ss.fit_transform(df_proc_test)

X_test = pd.DataFrame(X_test, columns=df_proc_test.columns)

# ##
X_test_red = pca.transform(X_test)
X_test_red.shape

## Testando modelos com o dataset inicial

In [ ]:
from sklearn.naive_bayes import GaussianNB
clf_NB = GaussianNB()
clf_NB.fit(X_train_red, df_y_train_winner)
clf_NB.score(X_test_red, df_y_test_winner)

In [ ]:
from sklearn.linear_model import Perceptron
clf_per = Perceptron(fit_intercept=False, max_iter=9000, tol=None, shuffle=True)
clf_per.fit(X_train_red, df_y_train_winner)
results = clf_per.predict(X_test_red)
clf_per.score(X_test_red, df_y_test_winner)

In [ ]:
import xgboost as xgb

model = xgb.XGBClassifier(n_estimators=300, max_depth=15)
model.fit(X_train_red, df_y_train_winner)
model.score(X_test_red, df_y_test_winner)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf_rfc = RandomForestClassifier(max_depth=3, random_state=5)
# clf_rfc = RandomForestClassifier(max_depth=1)
clf_rfc.fit(X_train_red, df_y_train_winner)
clf_rfc.score(X_test_red, df_y_test_winner)
#best = 47.55%

In [ ]:
# https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.VotingClassifier.html

In [ ]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
votingClf = VotingClassifier(estimators=[('nb',clf_NB), ('rf', clf_rfc)], voting='soft')
votingClf.fit(X_train_red, df_y_train_winner)
votingClf.score(X_test_red, df_y_test_winner)

In [ ]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
votingClf = VotingClassifier(estimators=[('nb',clf_NB), ('rf', clf_rfc), ('xgb', model)], voting='soft')
votingClf.fit(X_train_red, df_y_train_winner)
votingClf.score(X_test_red, df_y_test_winner)
#best = 46.89%

In [ ]:
# Para salvar meu modelo em um arquivo

# import pickle
# filename = 'finalized_model_pred_soccer_games.sav'
# pickle.dump(clf_rfc, open(filename, 'wb'))

## Criando modelos para prever número de gols para cada time e colocando novamente no modelo

Primeiro para os times mandantes

In [ ]:
from sklearn.naive_bayes import GaussianNB
clf_NB_home_goals = GaussianNB()
clf_NB_home_goals.fit(X_train_red, df_y_train_home_goals)
clf_NB_home_goals.score(X_test_red, df_y_test_home_goals)
# clf_NB_home_goals.predict(X_train_red)

In [ ]:
from sklearn.linear_model import Perceptron
clf_per_home_goals = Perceptron(fit_intercept=False, max_iter=9000, tol=None, shuffle=True)
clf_per_home_goals.fit(X_train_red, df_y_train_home_goals)
clf_per_home_goals.score(X_test_red, df_y_test_home_goals)

In [ ]:
import xgboost as xgb

xgb_home_goals = xgb.XGBClassifier(n_estimators=300, max_depth=15)
xgb_home_goals.fit(X_train_red, df_y_train_home_goals)
xgb_home_goals.score(X_test_red, df_y_test_home_goals)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf_rfc_home_goals = RandomForestClassifier(max_depth=3, random_state=5)
#clf_rfc_home_goals = RandomForestClassifier(max_depth=1)
clf_rfc_home_goals.fit(X_train_red, df_y_train_home_goals)
clf_rfc_home_goals.score(X_test_red, df_y_test_home_goals)

O modelo RandomForest teve um melhor desempenho, por isso irei utilizar ele para prever os gols

In [ ]:
home_goals_from_model_train = clf_rfc_home_goals.predict(X_train_red)
home_goals_from_model_test = clf_rfc_home_goals.predict(X_test_red)

Agora farei o mesmo, mas com os times visitantes

In [ ]:
from sklearn.naive_bayes import GaussianNB
clf_NB_away_goals = GaussianNB()
clf_NB_away_goals.fit(X_train_red, df_y_train_away_goals)
clf_NB_away_goals.score(X_test_red, df_y_test_away_goals)
# clf_NB_home_goals.predict(X_train_red)

In [ ]:
from sklearn.linear_model import Perceptron
clf_per_away_goals = Perceptron(fit_intercept=False, max_iter=9000, tol=None, shuffle=True)
clf_per_away_goals.fit(X_train_red, df_y_train_away_goals)
clf_per_away_goals.score(X_test_red, df_y_test_away_goals)

In [ ]:
import xgboost as xgb

xgb_away_goals = xgb.XGBClassifier(n_estimators=300, max_depth=15)
xgb_away_goals.fit(X_train_red, df_y_train_away_goals)
xgb_away_goals.score(X_test_red, df_y_test_away_goals)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf_rfc_away_goals = RandomForestClassifier(max_depth=3, random_state=5)
#clf_rfc_away_goals = RandomForestClassifier(max_depth=1)
clf_rfc_away_goals.fit(X_train_red, df_y_train_away_goals)
clf_rfc_away_goals.score(X_test_red, df_y_test_away_goals)

Novamente, a RandomForest obteve uma melhor performance e será utilizada para prever os gols dos visitantes

In [ ]:
away_goals_from_model_train = clf_rfc_away_goals.predict(X_train_red)
away_goals_from_model_test = clf_rfc_away_goals.predict(X_test_red)

## Preparando os novos datasets

In [ ]:
all_rows_pred_goals_train = []
for game in range(len(away_goals_from_model_train)):
    row = [home_goals_from_model_train[game],away_goals_from_model_train[game]]
    all_rows_pred_goals_train.append(row)
df_w_pred_goals_train = pd.DataFrame(all_rows_pred_goals_train, columns=['previsao_placar_home','previsao_placar_away'])

adicionando os gols que foram previstos pelo modelo no dataset pré normalização

In [ ]:
#wpg = with predict goals
df_wpg_train = pd.concat([df_proc.reset_index(drop=True), df_w_pred_goals_train.reset_index(drop=True)], axis=1)

agora falta => 
- fazer o dataset de teste
- normalizar
- passar no CPA
- Testar nos modelos de novo

In [ ]:
all_rows_pred_goals_test = []
for game in range(len(away_goals_from_model_test)):
    row = [home_goals_from_model_test[game],away_goals_from_model_test[game]]
    all_rows_pred_goals_test.append(row)
df_w_pred_goals_test = pd.DataFrame(all_rows_pred_goals_test, columns=['previsao_placar_home','previsao_placar_away'])

In [ ]:
df_wpg_test = pd.concat([df_proc_test.reset_index(drop=True), df_w_pred_goals_test.reset_index(drop=True)], axis=1)

Normalizar

In [ ]:
# Normalização do dataset
ss_2 = preprocessing.StandardScaler()
X_train_2 = ss_2.fit_transform(df_wpg_train)
X_train_2 = pd.DataFrame(X_train_2, columns=df_wpg_train.columns)

X_test_2 = ss_2.transform(df_wpg_test)
X_test_2 = pd.DataFrame(X_test_2, columns=df_wpg_test.columns)

In [ ]:
# Verificando a importancia de cada dimensão
pca_2 = PCA()
pca_2.fit(X_train_2)
features_2 = range(pca_2.n_components_)

fig_2 = go.Figure()
fig_2.add_trace(
    go.Scatter(
        x = list(features_2),
        y = 1 - pca_2.explained_variance_ratio_
    )
)

fig_2.show()

In [ ]:
pca_2 = PCA(n_components=288)
X_train_red_2 = pca_2.fit_transform(X_train_2)
print(X_train_red_2.shape)

X_test_red_2 = pca_2.transform(X_test_2)
print(X_test_red_2.shape)

## Testando nos modelos com os novos datasets criados

In [ ]:
clf_NB_2 = GaussianNB()
clf_NB_2.fit(X_train_red_2, df_y_train_winner)
clf_NB_2.score(X_test_red_2, df_y_test_winner)

In [ ]:
clf_per_2 = Perceptron(fit_intercept=False, max_iter=1000, tol=None, shuffle=True)
clf_per_2.fit(X_train_red_2, df_y_train_winner)
clf_per_2.score(X_test_red_2, df_y_test_winner)

In [ ]:
xgb_2 = xgb.XGBClassifier(n_estimators=300, max_depth=10)
xgb_2.fit(X_train_red_2, df_y_train_winner)
xgb_2.score(X_test_red_2, df_y_test_winner)

In [ ]:
clf_rfc_2 = RandomForestClassifier(max_depth=12, random_state=15)
clf_rfc_2.fit(X_train_red_2, df_y_train_winner)
clf_rfc_2.score(X_test_red_2, df_y_test_winner)